In [1]:
%cd ../

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
%matplotlib inline

In [4]:
INCLUDE_REST = False

In [5]:
if INCLUDE_REST:
    task_id = 0
else:
    task_id = 1    

In [6]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [7]:
if INCLUDE_REST:
    db_file = 'data/synthetic_data_multidistz_withrest_wholebrain.db'
else:
    db_file = 'data/synthetic_data_multidistz_norest_wholebrain.db'
   
mask_file = 'data/synthetic_data/wholebrain.nii.gz'

In [8]:
Group_1_last = 5
Group_2_last = 10

In [9]:
OVERWRITE = True
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

In [10]:
if not existed or OVERWRITE:
    data_files = utils.sorted_glob('data/synthetic_data/nifti_files/*.nii')
    regex = re.compile("subject_([0-9]{1,2})_task_([1-3])_stimulus_([0-9]{1,2}).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        [subject, task, stimulus] = m.groups()
        subject = int(subject)
        if int(task)>task_id:
            if subject >= 1 and subject <= Group_1_last:
                condition = 0
            elif subject > Group_1_last and subject <= Group_2_last:
                condition = 1
            else:
                condition = 2
            task = int(task)
            stimulus = int(stimulus)
            logging.info("Recording blocks in %s", data_file)

            synthetic_block = niidb.FMriActivationBlock(zscore=False)
            if task == 1:
                synthetic_block.task = 'rest_' + str(stimulus)
            elif task == 2:
                synthetic_block.task = 'baseline_' + str(stimulus)
            else:
                synthetic_block.task = 'task_' + str(stimulus)
            synthetic_block.filename = data_file
            synthetic_block.subject = subject
            synthetic_block.start_time = 0
            synthetic_block.end_time = None
            synthetic_block.individual_differences = {
                'condition': condition,
                'stimulus_group': task,
            }
            db.upsert(synthetic_block)
            logging.info('Subject %d performing task %s', subject, synthetic_block.task)

10/01/2019 17:54:55 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_2_stimulus_1.nii
10/01/2019 17:54:55 Subject 10 performing task baseline_1
10/01/2019 17:54:55 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_2_stimulus_2.nii
10/01/2019 17:54:55 Subject 10 performing task baseline_2
10/01/2019 17:54:55 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_2_stimulus_3.nii
10/01/2019 17:54:56 Subject 10 performing task baseline_3
10/01/2019 17:54:56 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_2_stimulus_4.nii
10/01/2019 17:54:56 Subject 10 performing task baseline_4
10/01/2019 17:54:56 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_2_stimulus_5.nii
10/01/2019 17:54:56 Subject 10 performing task baseline_5
10/01/2019 17:54:56 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_3_stimulus_1.nii
10/01/2019 17:54:56 Subject 10 performing task task_1
10/01/2019 17:54:56 Record

In [11]:
logging.info('Finished building NiiDb out of synthetic dataset')

10/01/2019 17:55:03 Finished building NiiDb out of synthetic dataset
